# Automatic import of Hamburg drinking fountains data to Wikidata
The following script downloads fountain data from Open Data swiss as per https://github.com/water-fountains/proximap/issues/307, compares it to existing fountains in Wikidata for the same region, and creates Wikidata Quickstatement commands to complete the entries in Wikidata. New entities are created if no matching fountains are found.

## Initialize environment

In [112]:
from datetime import datetime as dt
dtFmt = "%y%m%d_%H%M%S"
print (dt.now().strftime(dtFmt))
import pandas as pd
import io
import numpy as np
from urllib.request import urlopen
import json
from math import *
from platform import python_version
print("Python v "+python_version())
#https://github.com/paulhoule/gastrodon/issues/7 
from gastrodon import RemoteEndpoint,QName,ttl,URIRef,inline
from matplotlib import pyplot


191124_014436
Python v 3.6.5


In [113]:
#@prefix wikibase: <wikibase: <http://wikiba.se/ontology#> .
prefixes=inline("""
   @prefix wd: <http://www.wikidata.org/entity/> .
   @prefix wdt: <http://www.wikidata.org/prop/direct/> .
   @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
   @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
""").graph
endpoint=RemoteEndpoint(
   #"https://query.wikidata.org/sparql"
    "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
   ,prefixes=prefixes
)

## Load data

In [16]:
df = pd.read_csv("osmFountainsHamburg191123_135847.csv")

In [17]:
df.head()

,type,id,lat,lon,amenity,image,name,wikidata,wikipedia,drinking_water:type,...,emergency,lit,fee,name:de,name:en,check_date,opening_hours,fixme,level,note
0,node,257690357,53.551760,10.020930,fountain,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,268192970,53.549850,9.935483,fountain,http://commons.wikimedia.org/wiki/File:Hh-Stuh...,Stuhlmannbrunnen,Q2358953,de:Stuhlmannbrunnen,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,276284733,53.545274,9.971903,drinking_water,NaN,NaN,NaN,NaN,fountain,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,292561393,53.570260,9.980357,fountain,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,297703374,53.580721,9.704032,fountain,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Rename columns to make them easier to work with

In [19]:
# remove not needed columns
df = df.drop(columns=['type','id'])

In [20]:
# rename columns
df = df.rename(index=str, columns=
               {"lat": "Y"})

In [21]:
# rename columns
df = df.rename(index=str, columns=
               {"lon": "X"})

In [22]:
df.head()

,Y,X,amenity,image,name,wikidata,wikipedia,drinking_water:type,operator,wheelchair,...,emergency,lit,fee,name:de,name:en,check_date,opening_hours,fixme,level,note
0,53.551760,10.020930,fountain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,53.549850,9.935483,fountain,http://commons.wikimedia.org/wiki/File:Hh-Stuh...,Stuhlmannbrunnen,Q2358953,de:Stuhlmannbrunnen,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,53.545274,9.971903,drinking_water,NaN,NaN,NaN,NaN,fountain,Hamburg Wasser,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,53.570260,9.980357,fountain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,53.580721,9.704032,fountain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
len(df)

127

## Identify already existing fountains
### Query fountains from Wikidata

In [24]:
# Find the geographic extent of the data

buffer = 0.0003  # in degrees, corresponds to about 20-30 meters)
bounds = {
    'minX': df['X'].min() - buffer,
    'minY': df['Y'].min() - buffer,
    'maxX': df['X'].max() + buffer,
    'maxY': df['Y'].max() + buffer
}
print("bounds: ")
for key,value in bounds.items():
    print(key+ ": "+str(value))

bounds: 
minX: 9.663358500000001
minY: 53.425859700000004
maxX: 10.2964728
maxY: 53.731296900000004


In [121]:
# Query fountains (both water wells and fountains) from Wikidata within bounding box found above
# placeLabel 

query_string = """SELECT ?place ?placeLabel ?location ?date ?catalog_code ?catalogLabel ?operator ?water_supply_type
WHERE
{{
  # Enter coordinates
  SERVICE wikibase:box {{
    ?place wdt:P625 ?location .
    bd:serviceParam wikibase:cornerWest "Point({minX} {minY})"^^geo:wktLiteral.
    bd:serviceParam wikibase:cornerEast "Point({maxX} {maxY})"^^geo:wktLiteral.
  }} .
  # Is a water well or fountain or subclass of fountain
  FILTER (EXISTS {{ ?place wdt:P31/wdt:P279* wd:Q43483 }} || EXISTS {{ ?place wdt:P31/wdt:P279* wd:Q483453 }}).
  SERVICE wikibase:label {{
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],de" .
  }} 
  OPTIONAL {{ ?place p:P528 ?catalog_code.
            ?catalog_code pq:P972 ?catalog.}}
  OPTIONAL {{ ?place wdt:P571 ?date.}}
  OPTIONAL {{ ?place wdt:P5623 ?water_supply_type}}
  OPTIONAL {{ ?place wdt:P137 ?operator.}}
  OPTIONAL {{ ?place rdfs:placeLabel ?placeLabel}}
}}
  """.format(**bounds)

print(query_string)

SELECT ?place ?placeLabel ?location ?date ?catalog_code ?catalogLabel ?operator ?water_supply_type
WHERE
{
  # Enter coordinates
  SERVICE wikibase:box {
    ?place wdt:P625 ?location .
    bd:serviceParam wikibase:cornerWest "Point(9.663358500000001 53.425859700000004)"^^geo:wktLiteral.
    bd:serviceParam wikibase:cornerEast "Point(10.2964728 53.731296900000004)"^^geo:wktLiteral.
  } .
  # Is a water well or fountain or subclass of fountain
  FILTER (EXISTS { ?place wdt:P31/wdt:P279* wd:Q43483 } || EXISTS { ?place wdt:P31/wdt:P279* wd:Q483453 }).
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],de" .
  } 
  OPTIONAL { ?place p:P528 ?catalog_code.
            ?catalog_code pq:P972 ?catalog.}
  OPTIONAL { ?place wdt:P571 ?date.}
  OPTIONAL { ?place wdt:P5623 ?water_supply_type}
  OPTIONAL { ?place wdt:P137 ?operator.}
  OPTIONAL { ?place rdfs:placeLabel ?placeLabel}
}
  


In [122]:
# Perform query
query_result = endpoint.select(query_string)

In [123]:
#print(query_string)
print("\n\nTotal number of rows incl. duplicates "+str(len(query_result))+" size "+str(query_result.size))



Total number of rows incl. duplicates 130 size 1040


### Tidy up data

In [124]:
# Extract coordinates from Wikidata results

query_result['X'] = query_result['location'].apply(lambda l:float(l.split('(')[1].split(' ')[0]))
query_result['Y'] = query_result['location'].apply(lambda l:float(l.split(' ')[1].split(')')[0]))

In [125]:
query_result.head(100)

,place,placeLabel,location,date,catalog_code,catalogLabel,operator,water_supply_type,X,Y
0,wd:Q1454814,Minervabrunnen,Point(9.95166667 53.54583333),None,None,None,None,None,9.951667,53.545833
1,wd:Q814528,Behnbrunnen,Point(9.94194444 53.54805556),None,None,None,None,None,9.941944,53.548056
2,wd:Q2358953,Stuhlmannbrunnen,Point(9.93527778 53.54972222),None,None,None,None,None,9.935278,53.549722
3,wd:Q2651496,Alsterfontäne,Point(9.99495278 53.55461667),None,None,None,None,None,9.994953,53.554617
4,wd:Q18632417,Vierländerin-Brunnen,Point(9.99014 53.54766),None,None,None,None,None,9.990140,53.547660
...,...,...,...,...,...,...,...,...,...,...
95,wd:Q76308142,Q76308142,Point(10.2961728 53.4268728),None,None,None,None,None,10.296173,53.426873
96,wd:Q76308157,Q76308157,Point(9.9418923 53.548183),None,None,None,None,None,9.941892,53.548183
97,wd:Q76308173,Q76308173,Point(10.0831994 53.626006),None,None,None,None,None,10.083199,53.626006
98,wd:Q76308188,Q76308188,Point(9.9919829 53.5509746),None,None,None,None,None,9.991983,53.550975


In [128]:
# show duplicates
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html#pandas.DataFrame.duplicated

ids = query_result['placeLabel']
#ids = query_result['label']
duplicates = query_result[ids.isin(ids[ids.duplicated(keep=False)])]
dupli = duplicates.drop(columns=['catalog_code', 'catalogLabel', 'water_supply_type', 'date', 'catalogLabel', 'location', 'operator'])
dupli.sort_values(by=['placeLabel'],inplace=True)
print ("Duplicates: "+str(len(dupli))+"\n\n")
print(dupli.to_string())

Duplicates: 0


Empty DataFrame
Columns: [place, placeLabel, X, Y]
Index: []


In [129]:
def createMerge(dupli, extraText):
    #Create MERGE Command
    linesDup = []
    prevQ= ""
    prevX=0
    prevY=0
    for index, row in dupli.iterrows():
        
        # either create new or edit existing entity
        if row['X'] == prevX:
            if row['Y'] == prevY:
                lineDup = "MERGE\t"+prevQ[3:]+"\t"+row['place'][3:]+"\n"
                linesDup.append(lineDup)
            else:
                prevX=row['X']
                prevY=row['Y']
                prevQ=row['place']
        else:
            prevX=row['X']
            prevY=row['Y']
            prevQ=row['place']
            
    print("Merge commands"+extraText+" total: "+str(len(linesDup))+"\n\n")
    print(linesDup)
    
createMerge(dupli, "")    

Merge commands total: 0


[]


In [130]:
#write MERGE command to File
quickStatDupliFileName = "quickstatement_commands_Hamburg_drink_DUPLI_"+dt.now().strftime(dtFmt)+".txt"
with io.open(quickStatDupliFileName, "w", encoding='utf8') as f:
    f.writelines(linesDup)
print("wrote '"+quickStatDupliFileName+"' with "+str(len(linesDup))+" lines")

NameError: name 'linesDup' is not defined

In [37]:
# show duplicates STRICT
#idsS = query_result['placeLabel','X','Y'] #this does not work yet
#duplicatesS = query_result[ids.isin(ids[ids.duplicated(keep=False)])]
#dupliS = duplicatesS.drop(columns=['catalog_code', 'catalogLabel', 'water_supply_type', 'date', 'catalogLabel', 'location', 'operator'])
#dupliS.sort_values(by=['placeLabel'],inplace=True)
#print(dupliS.to_string())
#createMerge(dupliS, " - strict")  

In [132]:
# remove duplicate entries

# duplicate entries are caused when e.g. a fountain has catalog codes from two catalogs
query_result = query_result.drop_duplicates('place')
print("\n\nTotal number of rows without duplicates "+str(len(query_result)))



Total number of rows without duplicates 130


In [133]:
query_result.head(100)

,place,placeLabel,location,date,catalog_code,catalogLabel,operator,water_supply_type,X,Y
0,wd:Q1454814,Minervabrunnen,Point(9.95166667 53.54583333),None,None,None,None,None,9.951667,53.545833
1,wd:Q814528,Behnbrunnen,Point(9.94194444 53.54805556),None,None,None,None,None,9.941944,53.548056
2,wd:Q2358953,Stuhlmannbrunnen,Point(9.93527778 53.54972222),None,None,None,None,None,9.935278,53.549722
3,wd:Q2651496,Alsterfontäne,Point(9.99495278 53.55461667),None,None,None,None,None,9.994953,53.554617
4,wd:Q18632417,Vierländerin-Brunnen,Point(9.99014 53.54766),None,None,None,None,None,9.990140,53.547660
...,...,...,...,...,...,...,...,...,...,...
95,wd:Q76308142,Q76308142,Point(10.2961728 53.4268728),None,None,None,None,None,10.296173,53.426873
96,wd:Q76308157,Q76308157,Point(9.9418923 53.548183),None,None,None,None,None,9.941892,53.548183
97,wd:Q76308173,Q76308173,Point(10.0831994 53.626006),None,None,None,None,None,10.083199,53.626006
98,wd:Q76308188,Q76308188,Point(9.9919829 53.5509746),None,None,None,None,None,9.991983,53.550975


### Compute distances between fountains

In [134]:
# helper function to compute distances on the globe, returns distances in meters
def spherical_dist(pos1, pos2, r=6371000):
    pos1 = pos1 * np.pi / 180
    pos2 = pos2 * np.pi / 180
    cos_lat1 = np.cos(pos1[..., 0])
    cos_lat2 = np.cos(pos2[..., 0])
    cos_lat_d = np.cos(pos1[..., 0] - pos2[..., 0])
    cos_lon_d = np.cos(pos1[..., 1] - pos2[..., 1])
    return r * np.arccos(cos_lat_d - cos_lat1 * cos_lat2 * (1 - cos_lon_d))


# compute distances from each ODZ fountain to each Wikidata fountain
distances = spherical_dist(df[['X','Y']].values[:, None], query_result[['X','Y']].values)

### Identify nearest and second nearest matches for each osm hamburg fountain

In [135]:
# indexes of nearest fountains
nearest_idx = np.argmin(distances, axis=1).tolist()

# QID of nearest fountains
df['nearest_qid'] = query_result.iloc[nearest_idx]['place'].apply(lambda id:id[3:]).tolist()

# distance to nearest fountain
df['nearest_distance'] = np.min(distances, axis=1).tolist()


# then remove nearest
i_line=0
for i_col in nearest_idx:
    distances[i_line, i_col] = 100000
    i_line += 1
# find distance to second nearest
df['2nd_nearest_distance'] = np.min(distances, axis=1).tolist()

df.head(100)

,Y,X,amenity,image,name,wikidata,wikipedia,drinking_water:type,operator,wheelchair,...,note,nearest_qid,nearest_distance,2nd_nearest_distance,nearest_has_label_de,nearest_has_date,nearest_has_operator,nearest_has_code,nearest_has_water_type,match_found
0,53.551760,10.020930,fountain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Q76306851,0.0,382.406639,True,False,False,False,False,no match
1,53.549850,9.935483,fountain,http://commons.wikimedia.org/wiki/File:Hh-Stuh...,Stuhlmannbrunnen,Q2358953,de:Stuhlmannbrunnen,NaN,NaN,NaN,...,NaN,Q76306866,0.0,26.737057,True,False,False,False,False,no match
2,53.545274,9.971903,drinking_water,NaN,NaN,NaN,NaN,fountain,Hamburg Wasser,yes,...,NaN,Q76306883,0.0,351.807248,True,False,False,False,False,no match
3,53.570260,9.980357,fountain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Q76306899,0.0,525.430280,True,False,False,False,False,no match
4,53.580721,9.704032,fountain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Q76306914,0.0,701.790747,True,False,False,False,False,no match
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,53.442080,9.885816,drinking_water,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Q76308238,0.0,22.110551,True,False,False,False,False,no match
96,53.556267,9.978353,fountain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Q76308254,0.0,129.717196,True,False,False,False,False,no match
97,53.442161,9.885998,fountain,NaN,Sprudelstein,NaN,NaN,NaN,NaN,NaN,...,NaN,Q76308269,0.0,22.110551,True,False,False,False,False,no match
98,53.442489,9.884875,fountain,NaN,Sprudelstein,NaN,NaN,NaN,NaN,NaN,...,NaN,Q76308285,0.0,113.821602,True,False,False,False,False,no match


### Find out what information already exists for the nearest fountains

In [147]:
# does nearest have label in german?
df['nearest_has_label_de'] = (query_result.iloc[nearest_idx]['place'].apply(lambda p:p[3:]) != query_result.iloc[nearest_idx]['placeLabel']).tolist()

# does nearest have date?
df['nearest_has_date'] = query_result.iloc[nearest_idx]['date'].apply(lambda d:d is not None).tolist()

# does nearest have operator?
df['nearest_has_operator'] = query_result.iloc[nearest_idx]['operator'].apply(lambda id:id is not None).tolist()

# does nearest have catalog code?
df['nearest_has_code'] = query_result.iloc[nearest_idx]['catalog_code'].apply(lambda id:id is not None).tolist()

# does nearest have water type?
df['nearest_has_water_type'] = query_result.iloc[nearest_idx]['water_supply_type'].apply(lambda id:id is not None).tolist()

### Decide on whether nearest fountain should be considered a match

In [148]:
# The nearest fountain is a match if: 
# - no further than x m away
# - 2nd nearest fountain at nearest least ratio_min further away than the nearest fountain
def validate_proposal(qid, d1, d2, dmax=10, ratio_min=0.5):
    
    if d1 == 0 or (d1<=dmax and d2/d1-1 >= ratio_min):
        return 'match'
    elif d1<=dmax and d2/d1-1 < ratio_min:
        return 'unclear'
    else:
        print("Q-# "+qid+"\td1 "+str(d1)+"\td2 "+str(d2))
        return 'no match'
    
for index, row in df.iterrows():
    df.loc[index, 'match_found'] = validate_proposal(
        row['nearest_qid'], 
        row['nearest_distance'], 
        row['2nd_nearest_distance'],
        dmax=15
    )
dffinal = df.drop(columns=['nearest_distance', '2nd_nearest_distance'])

In [55]:
len(dffinal)

127

In [149]:
dffinal

,Y,X,amenity,image,name,wikidata,wikipedia,drinking_water:type,operator,wheelchair,...,fixme,level,note,nearest_qid,nearest_has_label_de,nearest_has_date,nearest_has_operator,nearest_has_code,nearest_has_water_type,match_found
0,53.551760,10.020930,fountain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Q76306851,False,False,False,False,False,match
1,53.549850,9.935483,fountain,http://commons.wikimedia.org/wiki/File:Hh-Stuh...,Stuhlmannbrunnen,Q2358953,de:Stuhlmannbrunnen,NaN,NaN,NaN,...,NaN,NaN,NaN,Q76306866,False,False,False,False,False,match
2,53.545274,9.971903,drinking_water,NaN,NaN,NaN,NaN,fountain,Hamburg Wasser,yes,...,NaN,NaN,NaN,Q76306883,False,False,False,False,False,match
3,53.570260,9.980357,fountain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Q76306899,False,False,False,False,False,match
4,53.580721,9.704032,fountain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Q76306914,False,False,False,False,False,match
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,53.623274,10.077671,drinking_water,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Q76308640,False,False,False,False,False,match
123,53.563470,10.073625,fountain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Q76308653,False,False,False,False,False,match
124,53.558327,9.823864,fountain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Q76308669,False,False,False,False,False,match
125,53.706815,9.683915,fountain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Q76308682,False,False,False,False,False,match


In [150]:
print("\n\nTotal number of rows "+str(len(dffinal)))



Total number of rows 127


## Create Quickstatement commands from data
### Helper functions to format content according to Quickstatements v1 syntax

In [191]:
def process_coordinates(x, y):
    # format geographic coordinates
    return '@{1:1.8f}/{0:1.8f}'.format(x,y)


def process_year(date):
    # format date
    if np.isnan(date):
        return ''
    else:
        return '+{0:4d}-00-00T00:00:00Z/9'.format(int(date))

    
fountain_type_map = {
    'öffentlicher Brunnen': 'Q53628296',
    'Notwasserbrunnen': 'Q53628522',
    'privater Brunnen': 'Q53629707',
    'Brunnen in städtischer Liegenschaft': 'Q53628618',
    'Brunnen des Verschönerungsvereins': 'Q53628761',
    'Brunnen mit eigener Versorgung': 'Q53630002'
}

water_type_map = {
    'Verteilnetz': 'Q53633635',
    'Quellwasser': 'Q1881858',
    'eigene Versorgung': 'Q53634173',
    'Grundwasser': 'Q161598'
}

def process_fountain_type(type):
    # translate fountain types to wikidata values
    return fountain_type_map[type]


def process_water_type(type):
    # translate water types to wikidata values
    return water_type_map[type]


def process_label_de(text):
    # process German language labels
    if text is None:
        return ''
    elif 'nan' == text.lower():
        return '"Brunnen"'.format(text)
    elif 'brunnen' in text.lower():
        return '"{}"'.format(text)
    elif 'fountain' in text.lower():
        return '"{}"'.format(text)
    else:
        return '"Brunnen ({})"'.format(text)
    

def createline(lines, item, prop, value, extra, qualifiers=[]):
    # general function to create Quickstatement v1 commands
    if value != '' and value != '""':
        statement = '{}\t{}\t{}'.format(item, prop, value)
        if len(qualifiers):
            # append qualifiers if applicable
            for q in qualifiers:
                statement += '\t{}\t{}'.format(q['prop'], q['value'])
        statement += extra
        statement += '\n'
        lines.append(statement)
    return lines

### Create statements, taking care not to overwrite existing data

In [208]:
# initialize command storage list
lines = []
statedId = "\tS248\tQ1224853"
i=0
for index, row in dffinal.iterrows():
    i+=1
    # either create new or edit existing entity
    if row['match_found'] == 'no match':
        # create a new fountain
        lines.append('CREATE\n')
        item = 'LAST'
    elif row['match_found'] == 'unclear':
        print('unclear match')
        print(row)
        continue
    elif row['match_found'] == 'match':
        # update existing fountain
        item = row['nearest_qid']
        
        
    # Add this basic information only if creating a new entity
    if item == 'LAST':
        # instance of  fountain
        lines = createline(lines, item, 'P31', 'Q483453',statedId)

        # coordinates
        lines = createline(lines, item, 'P625', process_coordinates(row['X'], row['Y']),statedId)
        
    # For other properties, add information if the entity is new or if property does not yet exist
    
    # label in german
    if item == 'LAST' or not row['nearest_has_label_de']:
        lines = createline(lines, item, 'Lde', process_label_de(str(row['name'])),statedId)
    # creation date
    #if item == 'LAST' or not row['nearest_has_date']:
    #    lines = createline(lines, item, 'P571', process_year(row['date']))

    # operated by  Hamburg Wasser
    #print(str(i)+": "+str(row['operator']))
    if  pd.isna(row['operator']):
        print(str(i)+": "+"operator none")
    else:
         lines = createline(lines, item, 'P137', 'Q1572943',statedId)
            
    # catalog number can always be added (it is hard to check for)
    #lines = createline(lines, item, 'P528', '"{}"'.format(row['operator_id']), [{
    #    'prop': 'P972',
    #    'value': 'Q53629101'
    #}])

1: operator none
2: operator none
4: operator none
5: operator none
6: operator none
8: operator none
9: operator none
10: operator none
11: operator none
12: operator none
13: operator none
14: operator none
15: operator none
16: operator none
17: operator none
18: operator none
19: operator none
20: operator none
21: operator none
22: operator none
23: operator none
24: operator none
25: operator none
26: operator none
27: operator none
28: operator none
29: operator none
30: operator none
31: operator none
32: operator none
33: operator none
34: operator none
35: operator none
36: operator none
37: operator none
38: operator none
39: operator none
40: operator none
41: operator none
42: operator none
44: operator none
45: operator none
46: operator none
47: operator none
48: operator none
49: operator none
50: operator none
51: operator none
52: operator none
53: operator none
54: operator none
55: operator none
56: operator none
57: operator none
58: operator none
59: operator none

# Write commands to file

In [209]:
quickStatFileName = "quickstatement_commands_Hamburg_fountain_"+dt.now().strftime(dtFmt)+".txt"
with io.open(quickStatFileName, "w", encoding='utf8') as f:
    f.writelines(lines)
print("wrote '"+quickStatFileName+"' with "+str(len(lines))+" lines")

wrote 'quickstatement_commands_Hamburg_fountain_191124_084735.txt' with 382 lines


# Import into Wikidata
- Go to https://tools.wmflabs.org/wikidata-todo/quick_statements.php.
- Authenticate yourself with your Wikidata account.
- Copy and paste the contents of quickstatement_commands*.txt into the blank field, and run the commands

see ../20191030_1600_import.png

...
58. Processing Q72935495 (Q72935495 Lde "Brunnen (Seelöwe-Planschbecken )")
59. Processing Q72935495 (Q72935495 P137 Q27229237)

All done!.

In [15]:
# it may well take half an hour until it works https://query.wikidata.org/